<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/transformer_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math



class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V):
        scale_factor = math.sqrt(self.head_dim)
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / scale_factor
        attention_weights = F.softmax(attention_scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        attention_scores = self.scaled_dot_product_attention(Q, K, V)

        concatenated_attention = attention_scores.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(concatenated_attention)

        return output


In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].detach()

In [12]:

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super(EncoderLayer, self).__init__()
        self.multihead_attention = MultiHeadAttention(d_model, num_heads)
        self.layer_norm = nn.LayerNorm(d_model)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )

    def forward(self, x):
        attention_output = self.multihead_attention(x, x, x)
        x = x + attention_output
        x = self.layer_norm(x)
        feedforward_output = self.feedforward(x)
        x = x + feedforward_output
        x = self.layer_norm(x)
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [13]:

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super(DecoderLayer, self).__init__()
        self.masked_multihead_attention = MultiHeadAttention(d_model, num_heads)
        self.multihead_attention = MultiHeadAttention(d_model, num_heads)
        self.layer_norm = nn.LayerNorm(d_model)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )

    def forward(self, x, encoder_output, mask=None):
        masked_attention_output = self.masked_multihead_attention(x, x, x, mask)
        x = x + masked_attention_output
        x = self.layer_norm(x)

        cross_attention_output = self.multihead_attention(x, encoder_output, encoder_output)
        x = x + cross_attention_output
        x = self.layer_norm(x)

        feedforward_output = self.feedforward(x)
        x = x + feedforward_output
        x = self.layer_norm(x)

        return x

class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads) for _ in range(num_layers)])

    def forward(self, x, encoder_output, mask=None):
        for layer in self.layers:
            x = layer(x, encoder_output, mask)
        return x


In [14]:
class Transformer(nn.Module):
    def __init__(self, d_model, num_heads, num_layers):
        super(Transformer, self).__init__()
        self.encoder = Encoder(d_model, num_heads, num_layers)
        self.decoder = Decoder(d_model, num_heads, num_layers)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        encoder_output = self.encoder(src)
        decoder_output = self.decoder(tgt, encoder_output, tgt_mask)
        return decoder_output

# Example usage:
d_model = 512
num_heads = 8
num_layers = 6
max_len = 100

# Instantiate the Transformer model
transformer = Transformer(d_model, num_heads, num_layers)

# Example input tensors
src = torch.rand((32, max_len, d_model))  # (batch_size, sequence_length, d_model)
tgt = torch.rand((32, max_len, d_model))  # (batch_size, sequence_length, d_model)

# Example forward pass
output = transformer(src, tgt)

print("Output shape:", output.shape)


Output shape: torch.Size([32, 100, 512])
